# chapter 5
## Using Decorators to Improve Our Code
In this chapter, we will explore decorators and see how they are useful in many 
situations where we want to improve our design. We will start by first exploring 
what decorators are, how they work, and how they are implemented.
With this knowledge, we will then revisit concepts that we learned in previous 
chapters regarding general good practices for software design and see how 
decorators can help us comply with each principle.

The goals of this chapter are as follows:
• To understand how decorators work in Python
• To learn how to implement decorators that apply to functions and classes
• To effectively implement decorators, avoiding common implementation 
mistakes
• To analyze how to avoid code duplication (the DRY principle) with 
decorators
• To study how decorators contribute to the separation of concerns
• To analyze examples of good decorators
• To review common situations, idioms, or patterns for when decorators 
are the right choice


## What are decorators in Python?
Decorators were introduced in Python a long time ago, in PEP-318, as a mechanism 
to simplify the way functions and methods are defined when they have to be 
modified after their original definition.

We first have to understand that in Python, functions are regular objects just like 
pretty much anything else. That means you can assign them to variables, pass them 
around by parameters, or even apply other functions to them. It was typical to want 
to write a small function, and then apply some transformations to it, generating a 
new modified version of that function (similar to how function composition works 
in mathematics).

One of the original motivations for introducing decorators was that because 
functions such as classmethod and staticmethod were used to transform the 
original definition of the method, they required an extra line, modifying the original 
definition of the function in a separate statement.

More generally speaking, every time we had to apply a transformation to a function, 
we had to call it with the modifier function, and then reassign it to the same name 
the function was originally defined with.

For instance, if we have a function called original, and then we have a function 
that changes the behavior of original on top of it, called modifier, we have to write 
something like the following:

In [ ]:
def original(...):
    ...
original = modifier(original)

Notice how we change the function and reassign it to the same name. This is 
confusing, error-prone (imagine that someone forgets to reassign the function, or 
does reassign it but not in the line immediately after the function definition but much 
farther away), and cumbersome. For this reason, some syntax support was added to 
the language

The previous example could be rewritten like so:

In [ ]:
@modifier
def original():
   ...

This means that decorators are just syntactic sugar for calling whatever is after 
the decorator as a first parameter of the decorator itself, and the result would be 
whatever the decorator returns.


The syntax for decorators improves the readability significantly, because now the 
reader of that code can find the whole definition of the function in a single place. 
Keep in mind that manually modifying functions as before is still allowed.

<div style="border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9; display: flex; align-items: center;">
    <div style="font-size: 24px; color: #555; padding-right: 10px;">💡</div>
    <div style="font-size: 14px;">
        <strong>Tip:</strong> 

In general, try to avoid re-assigning values to a function already 
designed without using the decorator syntax. Particularly, if the 
function gets reassigned to something else and this happens in a 
remote part of the code (far from where the function was originally 
defined), this will make your code harder to read.
    </div>

 


In line with the Python terminology, and our example, modifier is what we call the 
decorator, and original is the decorated function, often also called a wrapped object.

While the functionality was originally thought of for methods and functions, the 
actual syntax allows any kind of object to be decorated, so we are going to explore 
decorators applied to functions, methods, generators, and classes.

One final note is that while the name decorator is correct (after all, the decorator is 
making changes, extending, or working on top of the wrapped function), it is not to 
be confused with the decorator design pattern.

## Function decorators
Functions are probably the simplest representation of a Python object that can be 
decorated. We can use decorators on functions to apply all sorts of logic to them—we 
can validate parameters, check preconditions, change the behavior entirely, modify 
its signature, cache results (create a memoized version of the original function), and 
more.

As an example, we will create a basic decorator that implements a retry mechanism, 
controlling a particular domain-level exception and retrying a certain number of 
times:

In [2]:
# decorator_function_1.py
class ControlledException(Exception):
    """A generic exception on the program's domain."""
def retry(operation):
    a = 5
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        print(a)
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
    return wrapped

The use of @wraps can be ignored for now, as it will be covered in the Effective 
decorators – avoiding common mistakes section

<div style="border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9; display: flex; align-items: center;">
    <div style="font-size: 24px; color: #555; padding-right: 10px;">💡</div>
    <div style="font-size: 14px;">
        <strong>Tip:</strong> 

The use of _ in the for loop means that the number is assigned to 
a variable we are not interested in at the moment, because it's not 
used inside the for loop (it's a common idiom in Python to name _
values that are ignored)
    </div>

The retry decorator doesn't take any parameters, so it can be easily applied to any
function, as follows:

In [ ]:
@retry
def run_operation(task):
    """Run a particular task, simulating some failures on its execution."""
    return task.run()

The definition of @retry on top of run_operation is just syntactic sugar that Python 
provides to execute run_operation = retry(run_operation).

In this limited example, we can see how decorators can be used to create a generic 
retry operation that, under certain conditions (in this case, represented as exceptions 
that could be related to timeouts), will allow calling the decorated code multiple 
times

## Decorators for classes
Classes are also objects in Python (frankly, pretty much everything is an object in 
Python, and it's hard to find a counter-example; however, there are some technical 
nuances). That means the same considerations apply; they can also be passed 
by parameters, assigned to variables, asked some methods, or be transformed 
(decorated)

Class decorators were introduced in PEP-3129, and they have very similar 
considerations to the function decorators we just explored. The only difference 
is that when writing the code for this kind of decorator, we have to take into 
consideration that we are receiving a class as a parameter of the wrapped method, 
not another function

We have seen how to use a class decorator when we saw the dataclasses.dataclass
decorator in Chapter 2, Pythonic Code. In this chapter, we will learn how to write our 
own class decorators

Some practitioners might argue that decorating a class is something rather 
convoluted and that such a scenario might jeopardize readability because we would 
be declaring some attributes and methods in the class, but behind the scenes, the 
decorator might be applying changes that would render a completely different class.

This assessment is true, but only if this technique is heavily abused. Objectively, this 
is no different from decorating functions; after all, classes are just another type of 
object in the Python ecosystem, as functions are. We will review the pros and cons of 
this issue with decorators in the section titled Decorators and separation of concerns, but 
for now, we'll explore the benefits of decorators that apply particularly to classes:

#### • All the benefits of reusing code and the DRY principle. A valid case of a class decorator would be to enforce that multiple classes conform to a certain interface or criteria (by writing these checks only once in a decorator that is going to be applied to many classes).

#### • We could create smaller or simpler classes that will be enhanced later on by decorators.

#### • The transformation logic we need to apply to a certain class will be much easier to maintain if we use a decorator, as opposed to more complicated (and often rightfully discouraged) approaches such as meta-classes

Among all the possible applications of decorators, we will explore a simple example 
to show the sorts of things they can be useful for. Keep in mind that this is not the 
only application type for class decorators, and also that the code I show you can have 
many other solutions as well, all with their pros and cons, but I've chosen decorators 
for the purpose of illustrating their usefulness

Recalling our event systems for the monitoring platform, we now need to transform 
the data for each event and send it to an external system. However, each type of 
event might have its own particularities when selecting how to send its data

In particular, the event for a login might contain sensitive information such as 
credentials that we want to hide. Other fields such as timestamp might also require 
some transformations since we want to show them in a particular format. A first 
attempt at complying with these requirements would be as simple as having a class 
that maps to each particular event and knows how to serialize it:

In [ ]:
class LoginEventSerializer:
    def __init__(self, event):
        self.event = event
    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": "**redacted**",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M"),
        }
@dataclass
class LoginEvent:
    SERIALIZER = LoginEventSerializer
    username: str
    password: str
    ip: str
    timestamp: datetime
    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()

Here, we declare a class that is going to map directly with the login event, containing
the logic for it—hide the password field, and format the timestamp as required.

While this works and might look like a good option to start with, as time passes and 
we want to extend our system, we will find some issues:

• Too many classes: As the number of events grows, the number of 
serialization classes will grow in the same order of magnitude, because they 
are mapped one to one.

• The solution is not flexible enough: If we need to reuse parts of the 
components (for example, we need to hide the password in another type of 
event that also has it), we will have to extract this into a function, but also 
call it repeatedly from multiple classes, meaning that we are not reusing 
that much code after all.

• Boilerplate: The serialize() method will have to be present in all event 
classes, calling the same code. Although we can extract this into another 
class (creating a mixin), it does not seem like a good use of inheritance.


An alternative solution is to dynamically construct an object that, given a set of 
filters (transformation functions) and an event instance, can serialize it by applying 
the filters to its fields. We then only need to define the functions to transform each 
type of field, and the serializer is created by composing many of these functions.

Once we have this object, we can decorate the class in order to add the serialize()
method, which will just call these Serialization objects with itself:

In [ ]:
from dataclasses import dataclass
def hide_field(field) -> str:
    return "**redacted**"
def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")
def show_original(event_field):
    return event_field
class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields
    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation
            in self.serialization_fields.items()
        }
class Serialization:
    
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)
    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        event_class.serialize = serialize_method
        return event_class
@Serialization( 
    username=str.lower, 
    password=hide_field, 
    ip=show_original, 
    timestamp=format_time, 
) 
@dataclass 
class LoginEvent: 
    username: str 
    password: str 
    ip: str 
    timestamp: datetime

Notice how the decorator makes it easier for the user to know how each field is
going to be treated without having to look into the code of another class. Just by 
reading the arguments passed to the class decorator, we know that the username and 
IP address will be left unmodified, the password will be hidden, and the timestamp
will be formatted

Now, the code of the class does not need the serialize() method defined, nor 
does it need to extend from a mixin that implements it, since the decorator will add 
it. This is probably the only part that justifies the creation of the class decorator 
because otherwise, the Serialization object could have been a class attribute of 
LoginEvent, but the fact that it is altering the class by adding a new method to it 
makes it impossible

## Other types of decorator
Now that we know what the @ syntax for decorators means, we can conclude that it 
isn't just functions, methods, or classes that can be decorated; actually, anything that 
can be defined, such as generators, coroutines, and even objects that have already 
been decorated, can be decorated, meaning that decorators can be stacked.

The previous example showed how decorators can be chained. We first defined the 
class, and then applied @dataclass to it, which converted it into a data class, acting 
as a container for those attributes. After that, @Serialization will apply the logic to 
that class, resulting in a new class with the new serialize() method added to it.

Now that we know the fundamentals of decorators, and how to write them, we can 
move on to more complex examples. In the next sections, we'll see how to have more 
flexible decorators with parameters, and different ways of implementing them.

## More advanced decorators
With the introduction we've just had, we now know the basics of decorators: what 
they are, and their syntax and semantics

Now we're interested in more advanced uses of decorators that will help us structure 
our code more cleanly

We'll see that we can use decorators to separate concerns into smaller functions, and 
reuse code, but in order to so do effectively, we'd like to parametrize the decorators 
(otherwise, we'd end up repeating code). For this, we'll explore different options on 
how to pass arguments to decorators

After that, we can see some examples of good uses of decorators.

## Passing arguments to decorators
At this point, we already regard decorators as a powerful tool in Python. However, 
they could be even more powerful if we could just pass parameters to them so that 
their logic is abstracted even more

There are several ways of implementing decorators that can take arguments, but we 
will go over the most common ones. The first one is to create decorators as nested 
functions with a new level of indirection, making everything in the decorator fall 
one level deeper. The second approach is to use a class for the decorator (that is, to 
implement a callable object that still acts as a decorator).


In general, the second approach favors readability more, because it is easier to think 
in terms of an object than three or more nested functions working with closures. 
However, for completeness, we will explore both, and you can decide what is best 
for the problem at hand

## Decorators with nested functions
Roughly speaking, the general idea of a decorator is to create a function that returns 
another function (in functional programming, functions that take other functions as 
parameters are called higher-order functions, and it refers to the same concept we're 
talking about here). The internal function defined in the body of the decorator is 
going to be the one being called

Now, if we wish to pass parameters to it, we then need another level of indirection. 
The first function will take the parameters, and inside that function, we will define 
a new one, which will be the decorator, which in turn will define yet another new 
function, namely the one to be returned as a result of the decoration process. This 
means that we will have at least three levels of nested functions

Don't worry if this didn't seem clear so far. After reviewing the examples that are 
about to come, everything will become clear.

One of the first examples we saw of decorators implemented the retry 
functionality over some functions. This is a good idea, except it has a problem; our 
implementation did not allow us to specify the number of retries, and instead, this 
was a fixed number inside the decorator

Now, we want to be able to indicate how many retries each instance is going to have, 
and perhaps we could even add a default value to this parameter. In order to do this, 
we need another level of nested functions—first for the parameters, and then for the 
decorator itself

This is because we are now going to have something in the form of the following:

In [ ]:
@retry(arg1, arg2,... )

And that has to return a decorator because the @ syntax will apply the result of 
that computation to the object to be decorated. Semantically, it would translate to 
something like the following:

In [ ]:
<original_function> = retry(arg1, arg2, ....)(<original_function>)

Besides the number of desired retries, we can also indicate the types of exceptions we 
wish to control. The new version of the code supporting the new requirements might 
look like this:


In [ ]:
_DEFAULT_RETRIES_LIMIT = 3
def with_retry(
        retries_limit: int = _DEFAULT_RETRIES_LIMIT,
        allowed_exceptions: Optional[Sequence[Exception]] = None,
    ):
        allowed_exceptions = allowed_exceptions or
        (ControlledException,) # type: ignore
        def retry(operation):
            @wraps(operation)
            def wrapped(*args, **kwargs):
                last_raised = None
                for _ in range(retries_limit):
                    try:
                        return operation(*args, **kwargs)
                    except allowed_exceptions as e:
                        logger.warning(
                            "retrying %s due to %s",
                            operation.__qualname__, e
                        )
                        last_raised = e
                raise last_raised
            return wrapped
        return retry

Here are some examples of how this decorator can be applied to functions, showing 
the different options it accepts:

In [ ]:
# decorator_parametrized_1.py
@with_retry()
def run_operation(task):
    return task.run()
@with_retry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()
@with_retry(allowed_exceptions=(AttributeError,))
def run_with_custom_exceptions(task):
    return task.run()
@with_retry(
    retries_limit=4,8 allowed_exceptions=(ZeroDivisionError, AttributeError)
)
def run_with_custom_parameters(task):
    return task.run()

Using nested functions to implement decorators is probably the first thing we'd think 
of. This works well for most cases, but as you might have noticed, the indentation 
keeps adding up, for every new function we create, so soon it might lead to too many 
nested functions. Also, functions are stateless, so decorators written in this way 
won't necessarily hold internal data, as objects can

There's a different way of implementing decorators, which instead of using nested 
functions uses objects, as we explore in the next section

## Decorator objects
The previous example requires three levels of nested functions. The first is going to 
be a function that receives the parameters of the decorator we want to use. Inside this 
function, the rest of the functions are closures that use these parameters, along with 
the logic of the decorator

A cleaner implementation of this would be to use a class to define the decorator. In 
this case, we can pass the parameters in the __ init __ method, and then implement 
the logic of the decorator on the magic method named __ call __.


The code for the decorator will look like it does in the following example:

In [ ]:
_DEFAULT_RETRIES_LIMIT = 3
class WithRetry:
    def __init__(
        self,
        retries_limit: int = _DEFAULT_RETRIES_LIMIT,
        allowed_exceptions: Optional[Sequence[Exception]] = None,
    ) -> None:
    

     self.retries_limit = retries_limit
     self.allowed_exceptions = allowed_exceptions or
(ControlledException,)
def __call__(self, operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:logger.warning(
                    "retrying %s due to %s",
                    operation.__qualname__, e
                )
                last_raised = e
                raise last_raised
            return wrapped

And this decorator can be applied pretty much like the previous one, like so:

In [ ]:
@WithRetry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()

It is important to note how the Python syntax takes effect here. First, we create the 
object, so before the @ operation is applied, the object is created with its parameters 
passed to it. This will create a new object and initialize it with these parameters, as 
defined in the init method. After this, the @ operation is invoked, so this object will 
wrap the function named run_with_custom_retries_limit, meaning that it will be 
passed to the call magic method

Inside this call magic method, we defined the logic of the decorator as we normally 
do—we wrap the original function, returning a new one with the logic we want 
instead

## Decorators with default values
In the previous example, we saw a decorator that takes parameters, but those 
arguments have default values. The way the previous decorators were written will 
make sure that they work as long as users don't forget the parentheses to make the 
function call when using the decorator

For example, if we only wanted the default values, this would work:

In [ ]:
@retry()
def my function(): 

But this wouldn't:

In [ ]:
@retry
def my function(): ...

You might argue whether this is necessary and accept (perhaps with proper 
documentation) that the first example is how the decorator is meant to be used, 
and the second one is incorrect. And that would be fine, but it requires paying close 
attention, or runtime errors will occur

Of course, if the decorator takes parameters that don't have default values, then the 
second syntax doesn't make sense, and there's only one possibility, which might 
make things simpler.

Alternatively, you can make the decorator work with both syntaxes. As you might 
have guessed, this takes extra effort, and as always you should balance whether it's 
worth it.

Let's illustrate this with a simple example that uses a decorator with parameters 
to inject arguments into a function. We define a function that takes two parameters, 
and a decorator that does the same, and the idea is to call the function without 
arguments and let it work with the parameters passed by the decorator:

In [ ]:
@decorator(x=3, y=4)
def my_function(x, y):
            return x + y
my_function()  # 7

But of course, we define default values for the arguments of the decorator, so we 
can call it without values. And we'd like to also call it without the parentheses.

The simplest and most naïve way of writing it would be separating both cases with 
a conditional:

In [ ]:
def decorator(function=None, *, x=DEFAULT_X, y=DEFAULT_Y):
    if function is None:  # called as `@decorator(...)`
 
        def decorated(function):
            @wraps(function)
            def wrapped():
                return function(x, y)
 
            return wrapped
 
        return decorated
    else:  # called as `@decorator`
 
        @wraps(function)
        def wrapped():
            return function(x, y)
 
        return wrapped

Note something important about the signature of the decorator: the parameters are 
keyword-only. This simplifies the definition of the decorator a lot because we can 
assume the function is None when it's being called without arguments (otherwise, if 
we were to pass the values by position, the first of the parameters we passed would 
be confused with the function). If we wanted to be more careful, instead of using 
None (or any sentinel value), we could inspect the parameter type, assert a function 
object of the type we expect, and then shift the parameters accordingly, but that 
would make the decorator much more complicated.

Another alternative would be to abstract part of the wrapped decorator, and then 
apply a partial application of the function (using functools.partial). To explain 
this better, let's take an intermediate state, and use a lambda function that shows 
how the parameters to the decorator are applied, and how the decorator's arguments 
are "shifted":

In [ ]:
def decorator(function=None, *, x=DEFAULT_X, y=DEFAULT_Y):
    if function is None:
        return lambda f: decorator(f, x=x, y=y)
 
    @wraps(function)
    def wrapped():
        return function(x, y)
 
    return wrapped

This is analogous to the previous example, in the sense that we have the definition 
of the wrapped function (how it's being decorated). Then, if no function is provided, 
we return a new function that takes a function as a parameter (f) and returns the 
decorator with that function applied and the rest of the parameters bound. Then, in 
the second recursive call, the function will exist, and the regular decorator function 
(wrapped) will be returned instead.


You can achieve the same result by changing the lambda definition for a partial 
application of the function:

In [ ]:
return partial(decorator, x=x, y=y)

If this is too complex for our use cases, we can always decide to make the parameters 
of our decorators take mandatory values.

In any case, it's probably a good idea to define the parameters of the decorators 
to be keyword-only (regardless of whether they have default values or not). This 
is because, in general, when applying a decorator, there isn't that much context 
about what each value is doing, and using positional values might not yield a very 
meaningful expression, so it's better to be more expressive and pass the name of 
the parameter along with the value.


<div style="border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9; display: flex; align-items: center;">
    <div style="font-size: 24px; color: #555; padding-right: 10px;">💡</div>
    <div style="font-size: 14px;">
        <strong>Tip:</strong> 
       If you're defining decorators with arguments, prefer to make them 
keyword-only. 
</div>


Similarly, if our decorator does not intend to take parameters, and we want to be 
explicit about it, we can use the syntax we learned in Chapter 2, Pythonic Code, to 
define the function that our decorator receives as a single positional-only parameter.

For our first example, the syntax would be:

In [24]:
def retry(operation, /): ...

But keep in mind, this is not strictly recommended, just a way to make it explicit to 
you how the decorator is supposed to be invoked

## Decorators for coroutines
As explained in the introduction, since pretty much everything in Python is an 
object, then pretty much anything can be decorated, and this includes coroutines 
as well

However, there's a caveat here, and that is, as explained in previous chapters, 
asynchronous programming in Python introduces some differences in syntax. 
Therefore, these syntax differences will also be carried to the decorator.

Simply speaking, if we were to write a decorator for a coroutine, we could simply 
adapt to the new syntax (remember to await the wrapped coroutine and define the 
wrapped object as a coroutine itself, meaning the internal function will likely have 
to use 'async def' instead of just 'def').

The problem is if we want to have a decorator widely applicable to functions and 
coroutines. In most cases, creating two decorators would be the simplest (and 
perhaps best) approach, but if we wanted to expose a simpler interface for our users 
(by having fewer objects to remember), we could create a thin wrapper, acting like 
a dispatcher to two internal (not exposed) decorators. This would be like creating a 
facade but with a decorator

There is no general rule about how hard it would be to create a decorator for 
functions and coroutines, because that depends on the logic we want to put in the 
decorator itself. For example, in the code below there is a decorator that changes 
the parameters of the functions it receives, and this will work both for a regular 
function or a coroutine:

In [ ]:
X, Y = 1, 2
 
def decorator(callable):
    """Call <callable> with fixed values"""
 
    @wraps(callable)
    def wrapped():
        return callable(X, Y)
 
    return wrapped
 
@decorator
def func(x, y):
    return x + y
 
@decorator
async def coro(x, y):
    return x + y

It's important to make one distinction about the coroutine though. The decorator 
will receive the coroutine as its callable argument, and then invoke it with the 
parameters. This creates the coroutine object (the task that will go to the event loop), 
but it doesn't await it, meaning that whoever calls await coro() will end up awaiting 
the coroutine resulting from what the decorator wrapped. This means, in simple 
cases like this one, we don't need to replace the coroutine with another coroutine 
(although this is generally recommended).


But again, this depends on what we need to do. If we need a timing function, then 
we have to wait for the function or coroutine to complete to measure the time, and 
for this we'll have to call await on it, which means that the wrapper object will in 
turn have to be a coroutine (but not the main decorator, though).

The code below illustrates this example using a decorator that selectively decides 
how to wrap the caller function:

In [26]:
import inspect
def timing(callable):
    @wraps(callable)
    def wrapped(*args, **kwargs):
        start = time.time()
        result = callable(*args, **kwargs)
        latency = time.time() - start
        return {"latency": latency, "result": result}
 
    @wraps(callable)
    async def wrapped_coro(*args, **kwargs):
        start = time.time()
        result = await callable(*args, **kwargs)
        latency = time.time() - start
        return {"latency": latency, "result": result}
 
    if inspect.iscoroutinefunction(callable):
        return wrapped_coro
 
    return wrapped

The second wrapper is required for coroutines. If we didn't have it, then the code 
would have two problems. First, the call to callable (without await) would not 
actually wait for the operation to complete, meaning the results would be incorrect. 
And even worse, the value for the result key on the dictionary wouldn't be the 
result itself, but the coroutine created. As a consequence, the response would be a 
dictionary, and whoever tries to call that will try to await a dictionary, which will 
cause an error.

 

<div style="border: 1px solid #ccc; padding: 10px; background-color: #f9f9f9; display: flex; align-items: center;">
    <div style="font-size: 24px; color: #555; padding-right: 10px;">💡</div>
    <div style="font-size: 14px;">
        <strong>Tip:</strong> 
        As a general rule, you should replace a decorated object with 
another one of the same kind, that is, a function with a function, 
and a coroutine with another coroutine
</div>


We're still due to study one last enhancement that was recently added to Python, and 
it lifts some of the restrictions its syntax had.

## Extended syntax for decorators
Python 3.9 introduced a novelty for decorators, with PEP-614 (https://www.python.
org/dev/peps/pep-0614/), because a more general grammar is allowed. Before this 
enhancement, the syntax for invoking decorators (after the @) was restricted to very 
limited expressions, and not every single Python expression was allowed

With these restrictions lifted, we can now write more complex expressions and 
use them in our decorators, if we think that can save us some lines of code (but as 
always, be careful not to over-complicate and get a more compact but much harderto-read line).

As an example, we can simplify some of the nested functions we typically have for 
a simple decorator that logs a function call along with its parameters. Here (and 
for illustration purposes only), I replaced the nested function definitions, typical of 
decorators, with two lambda expressions:

In [1]:
def _log(f, *args, **kwargs):
    print(f"calling {f.__qualname__!r} with {args=} and {kwargs=}")
    return f(*args, **kwargs)
 
 
@(lambda f: lambda *args, **kwargs: _log(f, *args, **kwargs))
def func(x):
    return x + 1

In [ ]:
func(3)

The PEP document cites some examples for when this feature can be useful (like 
simplifying no-op functions to evaluate other expressions, or avoiding the use of 
the eval function).

The recommendation of this book for this feature is consistent with all the cases in 
which a more compact statement can be achieved: write the more compact version 
of the code as long as it doesn't hurt readability. If the decorator expression becomes 
hard to read, prefer the more verbose but simpler alternative of writing two or more 
functions.


## Good uses for decorators
In this section, we will take a look at some common patterns that make good use of 
decorators. These are common situations for when decorators are a good choice.

From all the countless applications decorators can be used for, we will enumerate 
a few, the most common or relevant ones:

#### • Transforming parameters: Changing the signature of a function to expose a nicer API, while encapsulating details on how the parameters are treated and transformed underneath. We must be careful with this use of decorators, because it's only a good trait when it's intentional. That means, if we are explicitly using decorators to provide a good signature for functions that had a rather convoluted one, then it's a great way of achieving cleaner code by means of decorators. If, on the other hand, the signature of a function changed inadvertently because of a decorator, then that's something we would want to avoid (and we'll discuss how toward the end of the chapter).

#### • Tracing code: Logging the execution of a function with its parameters. You might be familiar with multiple libraries that provide tracing capabilities, and often expose such functionality as decorators to add to our functions. This is a nice abstraction, and a good interface to provide, as a way of integrating the code with external parties without too much disruption. In addition, it's a great source of inspiration, so we can write our own logging or tracing functionality as decorators.

#### • Validating parameters: Decorators can be used to validate parameter types (against expected values or their annotations, for example) in a transparent way. With the use of decorators, we could enforce preconditions for our abstractions, following the ideas of designing by contract.

#### • Implementing retry operations: In a similar way to the example we've explored in the previous section.

#### • Simplifying classes by moving some (repetitive) logic into decorators: This is related to the DRY principle, which we'll revisit toward the end of the chapter

In the following sections, I'll discuss some of these topics in more detail.

## Adapting function signatures
In object-oriented design, sometimes there's the case of having objects with different 
interfaces that need to interact. A solution to this problem is the adapter design 
pattern, which we shall discuss in Chapter 7, Generators, Iterators, and Asynchronous 
Prpgramming, when we review some of the main design patterns.


The topic of this section is, however, similar, in the sense that sometimes we need to 
adapt not objects, but function signatures.

Imagine a scenario in which you're working with legacy code, and there's a module 
that contains lots of functions defined with a complex signature (lots of parameters, 
boilerplate, etc.). It would be nice to have a cleaner interface to interact with these 
definitions but changing lots of functions implies a major refactor.

We can use decorators to keep the differences in the changes to a minimum.

Sometimes we can use decorators as an adapter between our code and a framework 
we're using, if, for example, that framework has the aforementioned considerations.

Imagine the case of a framework that expects to call functions defined by us, 
maintaining a certain interface:

In [29]:
def resolver_function(root, args, context, info): ...

Now, we have that signature everywhere, and decide that it's better for us to create 
an abstraction from all those parameters that encapsulates them and exposes the 
behavior we need in our application

So now we have lots of functions whose first line repeats the boilerplate of creating 
the same object again and again, and then the rest of the function only interacts 
with our domain object:

In [30]:
def resolver_function(root, args, context, info):
    helper = DomainObject(root, args, context, info)
    ...
    helper.process()

In this example, we could have a decorator changing the signature of the function, 
so that we can write our functions assuming the helper object is passed directly. In 
this case, the task of the decorator would be to intercept the original parameters, 
create the domain object, and then pass the helper object to our function. We 
then define our functions assuming we only receive the object we need, already 
initialized.

Namely, we would like to write our code in this form:

In [ ]:
@DomainArgs
def resolver_function(helper):
    helper.process()


This works the other way around too, for example, if the legacy code we have is the 
one taking too many parameters, and we're always deconstructing an object that's 
already created, because refactoring the legacy code would be risky, then we can 
do that with a decorator as an intermediate layer.

The idea is that this use of decorators can help you write functions with simpler, 
more compact signatures

## Validating parameters
We have mentioned before that decorators can be used to validate parameters (and 
even enforce some preconditions or postconditions under the idea of Design by 
Contract (DbC)), so from this, you probably have got the idea that it is somewhat 
common to use decorators when dealing with or manipulating parameters.

In particular, there are some cases where we find ourselves repeatedly creating 
similar objects or applying similar transformations that we would wish to abstract 
away. Most of the time, we can achieve this by simply using a decorator.

## Tracing code
When talking about tracing in this section, we will refer to something more general 
that has to do with dealing with the execution of a function that we wish to monitor. 
This could refer to scenarios in which we want to:

• Trace the execution of a function (for example, by logging the lines it 
executes)

• Monitor some metrics over a function (such as CPU usage or memory 
footprint)

• Measure the running time of a function

• Log when a function was called, and the parameters that were passed to it

In the next section, we will explore a simple example of a decorator that logs the 
execution of a function, including its name and the time it took to run

## Effective decorators – avoiding common 
mistakes
While decorators are a great feature of Python, they are not exempt from issues if 
used incorrectly. In this section, we will see some common issues to avoid in order to 
create effective decorators.


## Preserving data about the original wrapped object
One of the most common problems when applying a decorator to a function is that 
some of the properties or attributes of the original function are not maintained, 
leading to undesired, and hard-to-track, side effects.

To illustrate this, we show a decorator that is in charge of logging when the function 
is about to run:

In [33]:
# decorator_wraps_1.py
def trace_decorator(function):
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)
    return wrapped

Now, let's imagine we have a function with this decorator applied to it. We might 
initially think that nothing of that function is modified with respect to its original 
definition:

In [34]:
@trace_decorator
def process_account(account_id: str):
    """Process an account by Id."""
    logger.info("processing account %s", account_id)
    ...

But maybe there are changes.

The decorator is not supposed to alter anything from the original function, but, as 
it turns out, since it contains a flaw, it's actually modifying its name and docstring, 
among other properties

Let's try to get help for this function:

In [ ]:
help(process_account)

And let's check how it's called:

In [ ]:
process_account.__qualname__

And also, the annotations for the original function were lost:

In [ ]:
process_account.__annotations__

We can see that, since the decorator is actually changing the original function for a 
new one (called wrapped), what we actually see are the properties of this function 
instead of those from the original function.

If we apply a decorator like this one to multiple functions, all with different names, 
they will all end up being called wrapped, which is a major concern (for example, if 
we want to log or trace the function, this will make debugging even harder).

Another problem is that if we placed docstrings with tests on these functions, they 
would be overridden by those of the decorator. As a result, the docstrings with the 
test we want will not run when we call our code with the doctest module (as we saw 
in Chapter 1, Introduction, Code Formatting, and Tools)

The fix is simple, though. We just have to apply the wraps decorator in the internal 
function (wrapped), telling it that it is actually a wrapping function:

In [45]:
# decorator_wraps_2.py
def trace_decorator(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)
    return wrapped

Now, if we check the properties, we will obtain what we expected in the first place. 
Check help for the function, like so:

In [ ]:
from decorator_wraps_2 import process_account
help(process_account)
Help on function process_account in module decorator_wraps_2:

process_account(account_id)
    Process an account by Id.

And verify that its qualified name is correct, like so:

In [ ]:
process_account.__qualname__

Most importantly, we recovered the unit tests we might have had on the docstrings! 
By using the wraps decorator, we can also access the original, unmodified function 
under the __wrapped__ attribute. Although it should not be used in production, it 
might come in handy in some unit tests when we want to check the unmodified 
version of the function

In general, for simple decorators, the way we would use functools.wraps would 
typically follow the following general formula or structure:

In [48]:
def decorator(original_function):
    @wraps(original_function)
    def decorated_function(*args, **kwargs):
        # modifications done by the decorator ...
        return original_function(*args, **kwargs)
    return decorated_function

Tips: Always use functools.wraps applied over the wrapped function, 
when creating a decorator, as shown in the preceding formula.


## Dealing with side effects in decorators
In this section, we will learn that it is advisable to avoid side effects in the body of the 
decorator. There are cases where this might be acceptable, but the bottom line is that 
if in doubt, decide against it, for the reasons that are explained ahead. Everything 
that the decorator needs to do aside from the function that it's decorating should 
be placed in the innermost function definition, or there will be problems when it 
comes to importing. Nonetheless, sometimes these side effects are required (or even 
desired) to run at import time, and the obverse applies

We will see examples of both, and where each one applies. If in doubt, err on the 
side of caution, and delay all side effects until the very last moment, right after the 
wrapped function is going to be called.

Next, we will see when it's not a good idea to place extra logic outside the wrapped
function.

## Incorrect handling of side effects in a decorator
Let's imagine the case of a decorator that was created with the goal of logging when 
a function started running and then logging its running time:

In [49]:
def traced_function_wrong(function):
    logger.info("started execution of %s", function)
    start_time = time.time()
    @wraps(function)
    def wrapped(*args, **kwargs):
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs",
            function,
            time.time() - start_time
        )
        return result
    return wrapped

Now we will apply the decorator to a regular function, thinking that it will work 
just fine:

In [ ]:
@traced_function_wrong
def process_with_delay(callback, delay=0):
    time.sleep(delay)
    return callback()

This decorator has a subtle yet critical bug in it

First, let's import the function, call it several times, and see what happens:

In [ ]:
from decorator_side_effects_1 import process_with_delay
INFO:started execution of <function process_with_delay at 0x...>

Just by importing the function, we will notice that something's amiss. The logging 
line should not be there, because the function was not invoked.

Now, what happens if we run the function, and see how long it takes to run? 
Actually, we would expect that calling the same function multiple times will give 
similar results:

In [ ]:
main()
...
INFO:function <function process_with_delay at 0x> took 8.67s
main()
...
INFO:function <function process_with_delay at 0x> took 13.39s
main()
...
INFO:function <function process_with_delay at 0x> took 17.01s

Every time we run the same function, it takes increasingly longer! At this point, you 
have probably already noticed the (now obvious) error.

Remember the syntax for decorators. @traced_function_wrong actually means the 
following:

In [ ]:
process_with_delay = traced_function_wrong(process_with_delay)

And this will run when the module is imported. Therefore, the time that is set in the 
function will be the time the module was imported. Successive calls will compute the 
time difference from the running time until that original starting time. It will also log 
at the wrong moment, and not when the function is actually called.

Luckily, the fix is also very simple—we just have to move the code inside the wrapped
function in order to delay its execution:

In [57]:
def traced_function(function):
    @functools.wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs",
            function.__qualname__,
            time.time() - start_time
        )
        return result
    return wrapped

With this new version, the previous problems are resolved.


If the actions of the decorator had been different, the results could have been much 
more disastrous. For instance, if it requires that you log events and send them to 
an external service, it will certainly fail unless the configuration has been run right 
before this has been imported, which we cannot guarantee. Even if we could, it 
would be bad practice. The same applies if the decorator has any other sort of side 
effect, such as reading from a file, parsing a configuration, and many more.

## Requiring decorators with side effects
Sometimes, side effects on decorators are necessary, and we should not delay their 
execution until the last possible minute, because that's part of the mechanism that is 
required for them to work

One common scenario for when we don't want to delay the side effect of decorators 
is when we need to register objects to a public registry that will be available in the 
module.

For instance, going back to our previous event system example, we now want to 
only make some events available in the module, but not all of them. In the hierarchy 
of events, we might want to have some intermediate classes that are not actual 
events we want to process on the system, but some of their derivative classes instead.

Instead of flagging each class based on whether it's going to be processed or not, 
we could explicitly register each class through a decorator

In this case, we have a class for all events that relate to the activities of a user. 
However, this is just an intermediate table for the types of events we actually want, 
namely UserLoginEvent and UserLogoutEvent

In [58]:
EVENTS_REGISTRY = {}
def register_event(event_cls):
    """Place the class for the event into the registry to make it     accessible in the module.
    """
    EVENTS_REGISTRY[event_cls.__name__] = event_cls
    return event_cls
class Event:
    """A base event object"""
class UserEvent:
    TYPE = "user"
@register_event
class UserLoginEvent(UserEvent):
    """Represents the event of a user when it has just accessed the system."""
@register_event
class UserLogoutEvent(UserEvent):
    """Event triggered right after a user abandoned the system."""

When we look at the preceding code, it seems that EVENTS_REGISTRY is empty, but 
after importing something from this module, it will get populated with all of the 
classes that are under the register_event decorator:

In [ ]:
from decorator_side_effects_2 import EVENTS_REGISTRY
EVENTS_REGISTRY
{'UserLoginEvent': decorator_side_effects_2.UserLoginEvent,
 'UserLogoutEvent': decorator_side_effects_2.UserLogoutEvent}

This might seem like it's hard to read, or even misleading, because EVENTS_REGISTRY
will have its final value at runtime, right after the module was imported, and we 
cannot easily predict its value by just looking at the code.

While that is true, in some cases this pattern is justified. In fact, many web 
frameworks or well-known libraries use this to work and expose objects or make 
them available. That said, be aware of this risk, if you are to implement something 
similar in your own projects: most of the time, an alternative solution would be 
preferred

It is also true that in this case, the decorator is not changing the wrapped object or 
altering the way it works in any way. However, the important note here is that if 
we were to do some modifications and define an internal function that modifies the 
wrapped object, we would still probably want the code that registers the resulting 
object outside it.

Notice the use of the word outside. It does not necessarily mean before, it's just not 
part of the same closure; but it's in the outer scope, so it's not delayed until runtime

## Creating decorators that will always work
There are several different scenarios to which decorators might apply. It can also 
be the case that we need to use the same decorator for objects that fall into these 
different multiple scenarios, for instance, if we want to reuse our decorator and 
apply it to a function, a class, a method, or a static method

If we create the decorator, just thinking about supporting only the first type of object 
we want to decorate, we might notice that the same decorator does not work equally 
well on a different type of object. A typical example is where we create a decorator 
to be used on a function, and then we want to apply it to a method of a class, only 
to realize that it does not work. A similar scenario might occur if we designed our 
decorator for a method, and then we want it to also apply for static methods or 
class methods.

When designing decorators, we typically think about reusing code, so we will want 
to use that decorator for functions and methods as well

Defining our decorators with the signature *args and **kwargs will make them 
work in all cases because it's the most generic kind of signature that we can 
have. However, sometimes we might want not to use this, and instead define the 
decorator-wrapping function according to the signature of the original function, 
mainly because of two reasons:

• It will be more readable since it resembles the original function.

• It actually needs to do something with the arguments, so receiving *args
and **kwargs wouldn't be convenient

Consider the case in which we have many functions in our code base that require 
a particular object to be created from a parameter. For instance, we pass a string, 
and initialize a driver object with it, repeatedly. Then we think we can remove the 
duplication by using a decorator that will take care of converting this parameter 
accordingly.

In the next example, we pretend that DBDriver is an object that knows how to connect 
and run operations on a database, but it needs a connection string. The methods we 
have in our code are designed to receive a string with the information of the database 
and require us to create an instance of DBDriver always. The idea of the decorator is 
that it's going to take the place of this conversion automatically—the function will 
continue to receive a string, but the decorator will create a DBDriver and pass it to 
the function, so internally we can assume that we receive the object we need directly.


An example of using this in a function is shown in the next listing:

In [ ]:
# src/decorator_universal_1.py
from functools import wraps
from log import logger
class DBDriver:
    def __init__(self, dbstring: str) -> None:
        self.dbstring = dbstring
    def execute(self, query: str) -> str:
        return f"query {query} at {self.dbstring}"
def inject_db_driver(function):
    """This decorator converts the parameter by creating a ``DBDriver``
    instance from the database dsn string.
    """
    @wraps(function)
    def wrapped(dbstring):
        return function(DBDriver(dbstring))
    return wrapped
@inject_db_driver
def run_query(driver):
    return driver.execute("test_function")

It's easy to verify that if we pass a string to the function, we get the result done by an 
instance of DBDriver, so the decorator works as expected:

In [ ]:
run_query("test_OK")
'query test_function at test_OK'

But now, we want to reuse this same decorator in a class method, where we find the 
same problem:

In [ ]:
class DataHandler:
    @inject_db_driver
    def run_query(self, driver):
        return driver.execute(self.__class__.__name__)

We try to use this decorator, only to realize that it doesn't work:

In [ ]:
DataHandler().run_query("test_fails")
Traceback (most recent call last):
  ...
TypeError: wrapped() takes 1 positional argument but 2 were given

What is the problem?

The method in the class is defined with an extra argument—self

Methods are just a particular kind of function that receive self (the object they're 
defined upon) as the first parameter.

Therefore, in this case, the decorator (designed to work with only one parameter, 
named dbstring) will interpret that self is said parameter, and call the method 
passing the string in the place of self, and nothing in the place of the second 
parameter, namely the string we are passing.

To fix this issue, we need to create a decorator that will work equally for methods 
and functions, and we do so by defining this as a decorator object that also 
implements the protocol descriptor

Descriptors are fully explained in Chapter 7, Generators, Iterators, and Asynchronous 
Programming, so for now, we can just take this as a recipe that will make the 
decorator work.

The solution is to implement the decorator as a class object and make this object 
a description, by implementing the __get__ method:

In [67]:
from functools import wraps
from types import MethodType
class inject_db_driver:
    """Convert a string to a DBDriver instance and pass this to the 
       wrapped function."""
    def __init__(self, function) -> None:
        self.function = function
        wraps(self.function)(self)
    def __call__(self, dbstring):
        return self.function(DBDriver(dbstring))
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.__class__(MethodType(self.function, instance))

Details on descriptors will be explained in Chapter 6, Getting More Out of Our Objects 
with Descriptors, but for the purposes of this example, we can now say that what it 
does is actually rebind the callable it's decorating to a method, meaning that it will 
bind the function to the object, and then recreate the decorator with this new callable

For functions, it still works, because it won't call the __ get __ method at all.

## Decorators and clean code
Now that we know more about decorators, how to write them, and avoiding 
common issues, it's time to take them to the next level and see how we can leverage 
what we have learned to achieve better software

We have briefly touched on this subject throughout the previous sections, but 
those were closer-to-the-code examples, as the suggestions referred to how to make 
specific lines (or sections) of the code more readable.

The topics discussed from now relate to more general design principles. Some of 
these ideas we have already visited in previous chapters, but the outlook here is 
to understand how we use decorators for such purposes

## Composition over inheritance
We have already discussed briefly that in general, it's better to have composition
rather than inheritance because the latter carries some problems of making 
components of the code more coupled

In the book Design Patterns: Elements of Reusable Object-Oriented Software (DESIG01), 
most of the ideas around the design pattern are based on the following idea:

  __Favor composition over class inheritance__

In Chapter 2, Pythonic Code, I introduced the idea of using the magic method 
__ getattr __ to resolve attributes dynamically on objects. I also gave the example 
that this could be used to automatically resolve attributes based on a naming 
convention should this be required by an external framework, for example. Let's 
explore two different versions of solving this problem.


For this example, let's assume we're interacting with a framework that has the 
naming convention of calling attributes with the prefix "resolve_" to resolve an 
attribute, but our domain objects only have those attributes without the "resolve_" 
prefix.

Clearly, we don't want to write a lot of repetitive methods named "resolve_x" for 
every attribute we have, so the first idea is to take advantage of the aforementioned 
  __ getattr __ magic method, and place it in a base class:

In [68]:
class BaseResolverMixin:
    def __getattr__(self, attr: str):
        if attr.startswith("resolve_"):
            *_, actual_attr = attr.partition("resolve_")
        else:
            actual_attr = attr
        try:
            return self.__dict__[actual_attr]
        except KeyError as e:
            raise AttributeError from e
 
 
@dataclass
class Customer(BaseResolverMixin):
    customer_id: str
    name: str
    address: str

This will do the trick, but can we do better?

We could devise a class decorator to set this method directly:

In [69]:
from dataclasses import dataclass
 
def _resolver_method(self, attr):
    """The resolution method of attributes that will replace __getattr__."""
    if attr.startswith("resolve_"):
        *_, actual_attr = attr.partition("resolve_")
    else:
        actual_attr = attr
    try:
        return self.__dict__[actual_attr]
    except KeyError as e:
        raise AttributeError from e
 
 
def with_resolver(cls):
    """Set the custom resolver method to a class."""
    cls.__getattr__ = _resolver_method
    return cls
 
 
@dataclass
@with_resolver
class Customer:
    customer_id: str
    name: str
    address: str

Both versions would comply with the following behavior:

In [ ]:
customer = Customer("1", "name", "address")
customer.resolve_customer_id

In [ ]:
customer.resolve_name

First, we have the resolve method as a standalone function that respects the signature 
of how the original __ getattr __ would look like (that's why I even preserved self
as the name of the first parameter, to be intentional about that function becoming a 
method)

The rest of the code seems rather simple. Our decorator only sets the method to the 
class it receives by parameter, and then we apply the decorator to our class without 
having to use inheritance anymore

How is this a little bit better than the previous example? For starters, we can argue 
the use of the decorator implies we're using composition (take a class, modify it, 
and return a new one) over inheritance, so our code is less coupled with the base 
class we had at the beginning.

Additionally, we can say that the use of inheritance (via a mixin class) in the first 
example was rather fictitious. We were not using inheritance to create a more 
specialized version of the class, only to take advantage of the __getattr__ method. 
This would be bad for two (complementary) reasons: first, inheritance is not the best 
way to reuse code. Good code is reused by having small, cohesive abstractions, not 
creating hierarchies. 


Second, remember from previous chapters that creating a subclass should follow 
with the idea of specialization, the "is a" kind of relation. Think whether, from a 
conceptual point of view, a customer is indeed a BaseResolverMivin (and what is, by 
the way?).

To shed some more light on this second point, imagine you have a hierarchy like 
this one:

In [72]:
class Connection: pass
class EncryptedConnection(Connection): pass

In this case, the use of inheritance is arguably correct, after all, an encrypted 
connection is a more specific kind of connection. But what would be a more specific 
kind of BaseResolverMixin? This is a mixin class, so it's expected to be mixed in 
the hierarchy along with other classes (using multiple inheritance). The use of this 
mix-in class is purely pragmatic, and for implementational purposes. Don't get me 
wrong, this is a pragmatic book, so you'll have to deal with mix-in classes in your 
professional experience, and it's perfectly fine to use them, but if we can avoid this 
purely implementational abstraction, and replace it with something that doesn't 
compromise our domain objects (in this case the Customer class), that's even better.

There's another exciting capability of the new design, which is extensibility. We 
have seen how decorators can be parametrized. Imagine the flexibility we could 
achieve in our design if we allowed the decorator to set any resolver function, not 
just the one we defined

## The DRY principle with decorators
We have seen how decorators allow us to abstract away certain logic into a separate 
component. The main advantage of this is that we can then apply the decorator 
multiple times to different objects in order to reuse code. This follows the Don't 
Repeat Yourself (DRY) principle since we define certain knowledge once and only 
once.

The retry mechanism implemented in the previous sections is a good example of a 
decorator that can be applied multiple times to reuse code. Instead of making each 
particular function include its own retry logic, we create a decorator and apply 
it several times. This makes sense once we have made sure that the decorator can 
work with methods and functions equally.

The class decorator that defined how events are to be represented also complies 
with the DRY principle in the sense that it defines one specific place for the logic for 
serializing an event, without needing to duplicate code scattered among different 
classes. Since we expect to reuse this decorator and apply it to many classes, its 
development (and complexity) pay off

This last remark is important to bear in mind when trying to use decorators in order 
to reuse code—we have to be absolutely sure that we will actually be saving code.

Any decorator (especially if it is not carefully designed) adds another level of 
indirection to the code, and hence more complexity. Readers of the code might 
want to follow the path of the decorator to fully understand the logic of the function 
(although these considerations are addressed in the following section), so keep in 
mind that this complexity has to pay off. If there is not going to be too much reuse, 
then do not go for a decorator and opt for a simpler option (maybe just a separate 
function or another small class is enough)

But how do we know what too much reuse is? Is there a rule to determine when 
to refactor existing code into a decorator? There is nothing specific to decorators in 
Python, but we could apply a general rule of thumb in software engineering (GLASS 
01) that states that a component should be tried out at least three times before 
considering creating a generic abstraction in the sort of a reusable component. From 
the same reference (GLASS 01) (I encourage all readers to read Facts and Fallacies of 
Software Engineering because it is a great reference) also comes the idea that creating 
reusable components is three times harder than creating simple ones.

The bottom line is that reusing code through decorators is acceptable, but only 
when you take into account the following considerations:

• Do not create the decorator in the first place from scratch. Wait until the 
pattern emerges and the abstraction for the decorator becomes clear, and 
then refactor.

• Consider that the decorator has to be applied several times (at least three 
times) before implementing it.

• Keep the code in the decorators to a minimum

As we have revisited the DRY principle from the point of view of decorators, we 
can still discuss the separation of concerns applied to decorators, as explored in 
the next section.

## Decorators and separation of concerns
The last point in the previous list is so important that it deserves a section of its 
own. We have already explored the idea of reusing code and noticed that a key 
element of reusing code is having components that are cohesive. This means that 
they should have the minimum level of responsibility—do one thing, one thing only, 
and do it well. The smaller our components, the more reusable, and the more they 
can be applied in a different context without carrying extra behavior that will cause 
coupling and dependencies, which will make the software rigid.


To show you what this means, let's reprise one of the decorators that we used in 
a previous example. We created a decorator that traced the execution of certain 
functions with code similar to the following:

In [73]:
def traced_function(function):
    @functools.wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs",
            function.__qualname__,
            time.time() - start_time
        )
        return result
    return wrapped

Now, this decorator, while it works, has a problem—it is doing more than one 
thing. It logs that a particular function was just invoked, and also logs how much 
time it took to run. Every time we use this decorator, we are carrying these two 
responsibilities, even if we only wanted one of them

This should be broken down into smaller decorators, each one with a more specific 
and limited responsibility:

In [74]:
def log_execution(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function.__qualname__)
        return function(*kwargs, **kwargs)
    return wrapped
def measure_time(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2f",
            function.__qualname__,
            time.time() - start_time,
        )
        return result
    return wrapped

Notice that the same functionality that we had previously can be achieved by simply 
combining both of them:

In [ ]:
@measure_time
@log_execution
def operation():
    ....

Notice how the order in which the decorators are applied is also important

Tips: Do not place more than one responsibility in a decorator. The 
single responsibility principle (SRP) applies to decorators as well

Finally, we can analyze good decorators to get an idea of how they're used in 
practice. The next section starts wrapping up what we've learned in this chapter, 
by analyzing decorators

## Analysis of good decorators
As a closing note for this chapter, let's review some examples of good decorators 
and how they are used both in Python itself, as well as in popular libraries. The 
idea is to get guidelines on how good decorators are created.

Before jumping into examples, let's first identify traits that good decorators should 
have:

• Encapsulation, or separation of concerns: A good decorator should 
effectively separate different responsibilities between what it does and what 
it is decorating. It cannot be a leaky abstraction, meaning that a client of the 
decorator should only invoke it in black-box mode, without knowing how it 
is actually implementing its logic.
    
• Orthogonality: What the decorator does should be independent, and as 
decoupled as possible from the object it is decorating.

• Reusability: It is desirable that the decorator can be applied to multiple 
types, and not that it just appears on one instance of one function, because 
that means that it could just have been a function instead. It has to be 
generic enough.

A nice example of decorators can be found in the Celery project, where a task is 
defined by applying the decorator of the task from the application to a function:

In [ ]:
@app.task
def mytask():
   ....

One of the reasons why this is a good decorator is because it is very good at 
something—encapsulation. The user of the library only needs to define the function 
body and the decorator will convert that into a task automatically. The @app.task
decorator surely wraps a lot of logic and code, but none of that is relevant to the 
body of mytask(). It is complete encapsulation and separation of concerns—nobody 
will have to take a look at what that decorator does, so it is a correct abstraction that 
does not leak any details.

Another common use of decorators is in web frameworks (Pyramid, Flask, and Sanic, 
just to name a few), on which the handlers for views are registered to the URLs 
through decorators:

In [ ]:
@route("/", method=["GET"])
def view_handler(request):
 ...

These sorts of decorators have the same considerations as before; they also provide 
total encapsulation because a user of the web framework rarely (if ever) needs to 
know what the @route decorator is doing. In this case, we know that the decorator is 
doing something more, such as registering these functions to a mapper to the URL, 
and also that it is changing the signature of the original function to provide us with a 
nicer interface that receives a request object with all the information already set.

The previous two examples are enough to make us notice something else about 
this use of decorators. They conform to an API. These libraries of frameworks 
are exposing their functionality to users through decorators, and it turns out that 
decorators are an excellent way of defining a clean programming interface

This is probably the best way we should think about decorators. Much like in the 
example of the class decorator that tells us how the attributes of the event are going 
to be handled, a good decorator should provide a clean interface so that users of 
the code know what to expect from the decorator, without needing to know how 
it works, or any of its details for that matter.


# Summary
Decorators are powerful tools in Python that can be applied to many things such 
as classes, methods, functions, generators, and many more. We have demonstrated 
how to create decorators in different ways, and for different purposes, and drew 
some conclusions along the way

When creating a decorator for functions, try to make its signature match the original 
function being decorated. Instead of using the generic *args and **kwargs, making 
the signature match the original one will make it easier to read and maintain, and 
it will resemble the original function more closely, so it will be more familiar to 
readers of that code.

Decorators are a very useful tool for reusing code and following the DRY principle. 
However, their usefulness comes at a cost, and if they are not used wisely, the 
complexity can do more harm than good. For that reason, we emphasize that 
decorators should be used when they are going to be applied multiple times 
(three or more times). In the same way as the DRY principle, we embrace the 
idea of separation of concerns, with the goal of keeping the decorators as small 
as possible.

Another good use of decorators is to create cleaner interfaces, for instance, 
simplifying the definition of a class by extracting part of its logic into a decorator. In 
this sense, decorators also help readability by providing the users with information 
about what that particular component will be doing, without needing to know how 
(encapsulation)

In the next chapter, we will take a look at another advanced feature of Python—
descriptors. In particular, we will see how with the help of descriptors, we can create 
even better decorators and solve some of the issues we encountered in this chapter.